## FEniCS Heat Example (example of unsteady simulation)
Observations:
* As nx increases, the errors decrease according to their respective convergence rates.
* This code uses its exact solution as its initial condition. (I'll try to use this in my code as well)

In [31]:
from __future__ import print_function
from fenics import *
import numpy as np

def compute_errors(u_e, u, t, mesh):
    u_e.t = t
    L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
    H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
    errors = {'L2 norm': L2n, 'H1 norm': H1n}
    return 'L2, ' + str(L2n) +', H1, '+ str(H1n) +', t, '+ str(t) +'\n'

def heat(N):
    T = 2.0            # final time
#     num_steps = 10     # number of time steps
    dt = 0.1 # time step size
    alpha = 3          # parameter alpha
    beta = 1.2         # parameter beta

    # Create mesh and define function space
    nx = ny = N
    mesh = UnitSquareMesh(nx, ny)
    V = FunctionSpace(mesh, 'P', 1)

    # Define boundary condition
    u_D = Expression('1 + x[0]*x[0] + alpha*x[1]*x[1] + beta*t',
                     degree=2, alpha=alpha, beta=beta, t=0)

    def boundary(x, on_boundary):
        return on_boundary

    bc = DirichletBC(V, u_D, boundary)

    # Define initial value
    u_n0 = interpolate(u_D, V)
    u_D.t += dt
    u_n1 = interpolate(u_D, V)
    #u_n = project(u_D, V)

    # Define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)
    f = Constant(beta - 2 - 2*alpha)

    # Backward Euler
#     F = (u-u_n)*v*dx 
#     F += dt*dot(grad(u), grad(v))*dx - dt*f*v*dx
    
    # BDF2
    F = (1.5*u - 2.0*u_n1 + 0.5*u_n0)*v*dx 
    F += dt*dot(grad(u), grad(v))*dx - dt*f*v*dx

    a, L = lhs(F), rhs(F)

    # Time-stepping
    u = Function(V)
    t = dt
#     for n in range(num_steps):
    while t - T < DOLFIN_EPS:
        # Update current time
        t += dt
        u_D.t = t

        # Compute solution
        solve(a == L, u, bc)

        # Plot solution
#         plot(u)

        # Compute error at vertices
#         u_e = interpolate(u_D, V)
#         error = np.abs(u_e.vector().array() - u.vector().array()).max()
#         print('t = %.2f: error = %.3g' % (t, error))

        # Update previous solution
        u_n0.assign(u_n1)
        u_n1.assign(u)
        
    errors = compute_errors(u_D, u, t, mesh)
    print(errors)

In [32]:
for N in [8, 16, 32, 64]:
    print("N = "+str(N)+"\n")
    heat(N)

N = 8

L2, 0.011048543456, H1, 0.228485018428, t, 2.0

N = 16

L2, 0.002762135864, H1, 0.114142291583, t, 2.0

N = 32

L2, 0.000690533966028, H1, 0.0570586117119, t, 2.0

N = 64

L2, 0.000172633491718, H1, 0.0285277388809, t, 2.0

